In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime

# Load datasets
category_data = pd.read_csv('Category_Data.csv')
department_data = pd.read_csv('Department_Data.csv')
store_data = pd.read_csv('Store_Data.csv')
sales_data = pd.read_csv('General Sales Data.csv')

# Data Cleaning
# Check for missing values
missing_values = sales_data.isnull().sum()
print("Missing Values:\n", missing_values)

# Handle missing values (replace with mean for numeric columns, mode for categorical columns)
sales_data['Sale Quantity'].fillna(sales_data['Sale Quantity'].mean(), inplace=True)
sales_data['Gross Value for single Product (Single Sale)'].fillna(sales_data['Gross Value for single Product (Single Sale)'].mean(), inplace=True)
# Repeat for other relevant columns...

# Identify and handle outliers
# Assume 'Sale Quantity' and 'Gross Value for single Product (Single Sale)' are relevant columns for outlier detection
sales_data = sales_data[(sales_data['Sale Quantity'] <= sales_data['Sale Quantity'].quantile(0.95)) & (sales_data['Gross Value for single Product (Single Sale)'] <= sales_data['Gross Value for single Product (Single Sale)'].quantile(0.95))]

# Ensure consistency in date formats
sales_data['SalesDate'] = pd.to_datetime(sales_data['SalesDate'])

# Feature Engineering
# Create new features: day of the week, month, year
sales_data['DayOfWeek'] = sales_data['SalesDate'].dt.dayofweek
sales_data['Month'] = sales_data['SalesDate'].dt.month
sales_data['Year'] = sales_data['SalesDate'].dt.year

# Merge datasets
sales_data = pd.merge(sales_data, category_data, on='Category ID', how='left')
sales_data = pd.merge(sales_data, department_data, on='Department ID', how='left')
sales_data = pd.merge(sales_data, store_data, on='Store ID', how='left')

# Data Transformation
# Apply normalization to relevant numerical features
scaler = StandardScaler()
numerical_features = ['Sale Quantity', 'Gross Value for single Product (Single Sale)', 'Retail Value for single Product (Single Sale)', 'Cost', 'Tax per unit', 'TotalQTY', 'TotalSales', 'Buying price per product']
sales_data[numerical_features] = scaler.fit_transform(sales_data[numerical_features])

# Encode categorical variables using Label Encoding
label_encoder = LabelEncoder()
categorical_features = ['Category Name', 'Department Name', 'Store Name', 'Location']
for feature in categorical_features:
    sales_data[feature] = label_encoder.fit_transform(sales_data[feature])

# Display the cleaned and transformed dataset
sales_data.head()


Missing Values:
 Department ID                                   0
Category ID                                     0
Store ID                                        0
Sale Quantity                                   0
Gross Value for single Product (Single Sale)    0
Retail Value (Single Sale)                      0
Tax with Revenue                                0
Cost                                            0
Tax per unit                                    0
TotalQTY                                        0
TotalSales                                      0
SalesDate                                       0
Sales Time                                      0
Buying price per product                        0
dtype: int64


ValueError: You are trying to merge on int64 and object columns for key 'Department ID'. If you wish to proceed you should use pd.concat